# DT Assignment

# Data Loading

In [7]:
import pandas as pd 
import numpy as np

In [8]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [98]:
def get_gini(df, label):
    a=0 #1에서 빼줄 값을 a에 담을 것이다. 
    count=len(df) #전체 길이(개수)
    label_df=df[label].value_counts() #각각 no,yes가 몇개인지 
    for i in range(len(label_df)):
        a+=(label_df[i]/count)**2
    gini=1-a 
        
    return gini

In [99]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [110]:
from itertools import combinations

def get_binary_split(df, attribute):
    result=[] #값을 담을 리스트 
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    for i in range(len(uniques)-1): #속성들을 전부반환하는 리스트는 만들지 않을 것이므로 -1
        list1=list(combinations(uniques,i+1)) #조합
        for j in range(len(list1)):
            list1[j]=list(list1[j]) #각 조합의 튜플을 리스트로 변경 
            result.append(list1[j]) #result리스트에 append
    return result

In [111]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [114]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {} #result를 담을 딕셔너리 
    binary_split = get_binary_split(df, attribute) #이진분류로 특성을 나누어준다. 
    n = len(df) #전체길이 
    for i in range(len(binary_split)):
        j=len(binary_split[i]) #분류한것의 첫번째(i=0) 리스트의 길이 
        a=binary_split[i][0:j] #분류한 것의 첫번째(i=0)안에 담긴 값들 담기 
        df_1=pd_data[pd_data[attribute].isin(a)] #a에 담긴 특성만 빼서 데이터프레임만들기 
        df_up=pd_data[~pd_data[attribute].isin(a)] #a가 담기지 않은 특성만 빼서 데이터프레임 만들기 
        gini1=get_gini(df_1,label) #각각 데이터프레임에서 지니계수구하기 
        gini2=get_gini(df_up,label)
        total_gini=(len(df_1)/n)*gini1 + (len(df_up)/n)*gini2 #최종 지니계수 구하기 
        k='_'.join(a)
        result[k]=total_gini
    
    return result

In [116]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [117]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4428571428571429)

In [118]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [120]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    k=min(get_attribute_gini_index(pd_data, feature, label).items())[1]
    print("Minimum Gini index of",feature,':',round(k,4))



Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [121]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youth_middle_aged': 0.45714285714285713,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [209]:
from math import log2

def getEntropy(df, feature) :
    a=0
    count=len(df)
    label_df=df[feature].value_counts()
    for i in range(len(label_df)):
        a-=(label_df[i]/count)*log2(label_df[i]/count) #지니계수만드는 함수랑 비슷하지만 최종 산출 식만 다름 
    return a


In [210]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [248]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
    result = {}
    new_columns=pd_data.columns.difference([feature]) #feature값만 제외한 columns들 :new_columns
    for i in range(len(new_columns)):
        A=pd_data[[new_columns[i],feature]] #new_columns의 값 한개씩과 feature만을 빼서 데이터프레임만들기 
        count=len(A)
        label=A[new_columns[i]].value_counts() #new_columns에서 나누어지는 특성별 값 개수보기 
        hui=0
        for j in range(len(label)): #label의 개수만큼 
            num=label[j] 
            goal=A[A[new_columns[i]]==label.index[j]]
            goal_return=(num/count)*(getEntropy(goal,feature))
            hui+=goal_return
        result_hui=getEntropy(df,feature)-hui
        result[new_columns[i]]=result_hui

    return result

In [249]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'credit_rating': 0.04812703040826949,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159}